In [1]:
# Importación de librerias y uso de Api key de Cohere
import os
from dotenv import load_dotenv
import cohere

load_dotenv()  # Load .env file

COHERE_API_KEY = os.getenv("API_KEY")
# print(COHERE_API_KEY)  # Verify the key

co = cohere.ClientV2(COHERE_API_KEY)

In [2]:
from pypdf import PdfReader
## Extracción de texto de archivo PDF.

# Función para extraer el texto de un archivo pdf
def extract_text_from_pdf(file_path):
    """
        1-Extrae el texto de las páginas del archivo.pdf
        2-Parámetros
            - file_path(String): Ruta del archivo PDF
        3-Retorna
            -combined_text(String) : texto de todas las páginas
            -page_texts(list) : Lista con el texto de cada una de las páginas.
    """
    try:
        # Cargar el documento pdf con el nombre de su ruta
        reader = PdfReader(file_path)
        
        # Extraer texto de todas las páginas
        page_texts = [page.extract_text() for page in reader.pages]

        # Combinar todas las páginas en un solo texto
        combined_text = "\n".join(page_texts)

        return combined_text, page_texts
        
    except FileNotFoundError:
        # Manejo del error en caso que  no exista el archivo
        print(f"Error: el archivo solicitado '{file_path}' no fue encontrado.")
        return None, None
    except Exception as e:
        # Manejo de cualquier otro tipo de error
        print(f"Error al leer el archivo PDF: {e}")
        return None, None

# Ejemplo de uso de la funcion -extract_text_from_pdf- para probar la extracción de texto
pdf_path = 'historias.pdf' #Ruta del archivo
combined_text, page_texts = extract_text_from_pdf(pdf_path)

if combined_text:
    print("Texto combinado del PDF:")
    # Mostrar solo una parte del texto combinado para evitar sobrecarga en la salida
    print(combined_text[:200])  #Ej: Muestra los primeros 200 caracteres
    print("\nNúmero de páginas:", len(page_texts)) #Número total de páginas
else:
    print("No fue posible extraer el texto del archivo PDF")

Texto combinado del PDF:
Sol y Luna 
Sol y Luna eran dos pequeños gatitos que habían nacido en la misma camada, pero sus 
personalidades no podían ser más diferentes. Sol, de un brillante color anaranjado, era 
aventurero y c

Número de páginas: 3


In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Primer paso: Configuración de la división del texto en chunks utilizando
# RecursiveCharacterTextSplitter de Langchain

text_splitter= RecursiveCharacterTextSplitter(
    chunk_size= 400, #Define el número de máximo de caracteres por chunk,Con 400 caracteres, mantiene oraciones completas, coherentes, y para   #ser manejada por algoritmos de busqueda como en este caso(generación de vector) para almacenar en ChromaDB.
    chunk_overlap= 50 #Indica la Superposición de 50 caracteres entre chunks, este valor garantiza que las transiciones entre ellos sean suaves, asegurando que no se pierda información entre cada solapamiento.
)

# Paso 2: Texto para dividir en chunks, utilizo el texto combinado obtenido de la extracción del pdf
historia= combined_text

# Paso 3: Crear los chunks(con previa validación) a partir del texto con la configuración ya definida.
if not historia.strip():
    print("El texto está vacío o no contiene contenido relevante para dividir.")
else:
    chunks = text_splitter.split_text(historia)

# Paso 4: Obtener los resultados impresos en pantalla

print(f"Historia dividida en {len(chunks)} chunks:")
for i, chunk in enumerate(chunks[:3]): #Ej, mostrar una cantidad limitada para no sobrecargar la salida
    print(f"Chunk {i + 1} ({len(chunk)} caracteres):\n{chunk}\n")


Historia dividida en 17 chunks:
Chunk 1 (356 caracteres):
Sol y Luna 
Sol y Luna eran dos pequeños gatitos que habían nacido en la misma camada, pero sus 
personalidades no podían ser más diferentes. Sol, de un brillante color anaranjado, era 
aventurero y curioso; siempre buscando nuevas experiencias y explorando cada rincón de 
su hogar. Luna, en cambio, era de un suave pelaje gris y tenía un temperamento más

Chunk 2 (310 caracteres):
sereno y observador. Pasaba horas contemplando el mundo desde la ventana, como si en 
cada sombra descubriera un misterio oculto. 
Una tarde, mientras Sol correteaba por el jardín persiguiendo mariposas, Luna permanecía 
en el alféizar, vigilando desde lejos. De repente, una fuerte tormenta comenzó a formarse

Chunk 3 (318 caracteres):
en el horizonte. El viento sacudía las ramas de los árboles, y las gotas de lluvia empezaron 
a caer pesadamente. Sol, atrapado por su espíritu inquieto, no se dio cuenta de lo rápido 
que se acercaba la tormenta. Luna, 

In [4]:
from langchain_chroma import Chroma
from langchain.embeddings import CohereEmbeddings

# Paso 1: Generación de embeddings para los chunks

# Utilicé el modelo de embed-multilingual-v3.0 para crear las representaciones vectoriales
# de cada chunk de texto. Estos embeddings se utilizaran para futuras búsquedas.

response = co.embed(
    texts= chunks, #Lista de chucnks para procesar
    model= "embed-multilingual-v3.0", # Modelo de embeddings multilingüe
    input_type="search_document",
    embedding_types=["float"],
)
# Paso 2: Obtención de los embeddings generados, se extraen como vectores flotantes.
embeddings= response.embeddings.float_

# Verificar la generación de los embeddings
if embeddings and len(embeddings) == len(chunks):
     print(f"Generados {len(embeddings)} embeddings para los {len(chunks)} chunks.")
else:
      print("Error: No fue posible generar los embeddings o el número de embeddings no coincide con los chunks.")



Generados 17 embeddings para los 17 chunks.


In [5]:
# Importación de libreria para manejar archivos JSON
import json 

# Paso 1: Asociar cada chunk con su embedding correspondiente
# Lista de diccionarios donde se encuentra el texto del chunk y su vector correspondiente.
data = [{"id": i + 1, "chunk": chunk, "embedding": embedding} for i, (chunk, embedding) in enumerate(zip(chunks, embeddings))]

# Paso 2: Extraer por separado de cada los chunks y embeddings para uso posterior
documents = [item["chunk"] for item in data]  # Extraer los textos (chunks)
embeddings = [item["embedding"] for item in data]  # Extraer los embeddings

# Paso 3: Guardar los datos en un archivo JSON para tener un acceso facil y reutilizable,
# previamente validando que la cantidad de chunks y embeddings sean iguales.
if len(chunks) != len(embeddings):
    print("Error: El número de chunks no coincide con el número de embeddings.")
else:
   try:
       with open("embeddings.json", "w") as f:
           json.dump(data, f)
           print("Embeddings y chunks guardados exitosamente.")
   except Exception as e:
        print(f"Error al guardar los embeddings: {e}")

Embeddings y chunks guardados exitosamente.


In [6]:
# Importación de Chromadb.
import chromadb
# Creación de un cliente para realizar la conexión a la base de datos vectorial
chroma_client = chromadb.Client()

# Crear o recuperar si ya existe una colección en ChromaDB (con el método get_or_create_collection)
collection_name = "text_chunks"
collection = chroma_client.get_or_create_collection(collection_name)

# Verificar la correspondencia entre documentos y embeddings
if len(documents) == len(embeddings):

    # Generar IDs dinámicamente segun la cantidad de chunks
    ids = [f"id_{i}" for i in range(len(documents))]
    try:
        collection.add(
            documents=documents,  # Lista de textos(chunks)
            embeddings=embeddings,  # Lista de embeddings correspondientes
            ids=ids  # Lista de IDs generados
        )
        print(f"Documentos y embeddings añadidos correctamente a la colección '{collection_name}'.")
    except Exception as e:
        print(f"Error al añadir documentos y embeddings a la colección: {e}")
else:
    # Manejo del error en caso que las longitudes sean diferentes
    print(f"Error: La cantidad de chunks ({len(documents)}) no coincide con la cantidad de embeddings ({len(embeddings)}).")
    
# Mostrar la colección para confirmar el estado
print(f"Estado de la colección '{collection_name}': {collection}")

Documentos y embeddings añadidos correctamente a la colección 'text_chunks'.
Estado de la colección 'text_chunks': Collection(name=text_chunks)


In [25]:
## Funciones independientes para generación de embeddings de prompts, búsqueda en base de datos y generación de respuesta.

# Función para generar el embedding para el prompt

def generar_embedding_prompt(prompt_text):
    """
        -La función genera el embedding para un texto dado utilizando el modelo de Cohere
        -Parámetros prompt_text(String):Texto para el cual se generará el embedding.
        -La función retorna el embedding generado como una lista de elementos flotantes.
    """
    try:
        response = co.embed(
            texts=[prompt_text],
            model="embed-multilingual-v3.0",
            input_type="search_document",
            embedding_types=["float"]
        )
        return response.embeddings.float_[0]
    except Exception as e:
        print(f"Error al generar el embedding del prompt: {e}")
        return "Hubo un problema al generar los embeddings. Intenta nuevamente."

# Funcion para búsqueda en base de datos Chromadb
def busqueda_en_chromadb(prompt_text):
    """
        -La función busca documentos relevantes en ChromaDB utilizando el embedding del prompt.
        -Parámetros prompt_text(String):Texto del prompt a buscar.
        -La función retorna el el documento relevante más cercando al embedding del prompt, 
        o retorna None si no encuentra un documento relevante.
    """
    try:
        # Generar el embedding del prompt
        prompt_embedding = generar_embedding_prompt(prompt_text)
        if not prompt_embedding:
            return None
        
        # Consulta en la colección en ChromaDB.
        resultados = collection.query(
            query_embeddings=[prompt_embedding],  # Embedding del prompt
            n_results=1  # Número de resultados a recuperar
        )
         # Devolver el documento más relevante si existe
        return resultados['documents'][0] if resultados['documents'] else None
    except Exception as e:
        print(f"Error al buscar en ChromaDB: {e}")
        return None
    
# Función para generar una respuesta final utilizando Cohere     
def generar_respuesta_final(documento_relevante, pregunta, system_prompt):
    """
        -La función genera una respuesta basada en el contenido de un documento y un prompt proporcionado.
        -Parámetros:
            documento(String): Texto relevante recuperado de la base de datos.
            pregunta (String Pregunta del usuario.
            system_prompt (String): Instrucciones para el modelo de generación de texto.
        - Respuesta generada o mensaje de error si no hay documentos relevantes.
    """
    
    if documento_relevante:
        try:
            # Preparar el prompt para la generación de respuesta
            prompt_con_instrucciones = f"{system_prompt}\nTexto relevante: {documento_relevante}\n\nPregunta: {pregunta}"
            
            #Generación de la respuesta utilizando un modelo de Cohere
            response = co.generate(
                model="command-r-plus-08-2024", 
                prompt=prompt_con_instrucciones,
                max_tokens=100,
                temperature=0.3, 
            )
            return response.generations[0].text.strip() 
        except Exception as e:
            print(f"Error al generar la respuesta con Cohere: {e}")
            return "Hubo un problema al generar la respuesta. Por favor, intenta nuevamente."
    else:
        return "No se encontraron resultados relevantes para tu pregunta."
        
# Definir el system prompt con las instrucciones
system_prompt = """
Tu trabajo es responder a las preguntas, con las siguientes características:
- Responde de manera amigable y con tono entusiasta, como si le hablaras a un niño.
- Responde en máximo 3 oraciones.
- Agrega emojis a la respuesta.
- Ante la misma pregunta debes responder lo más similar posible para cada interacción.
- Responde siempre en español, sin importar en qué idioma se haga la pregunta.
- Solo debes utilizar el contenido de las historias para responder sobre las preguntas del usuario.
"""


In [26]:
## Implementación de RAG_answer
# Función para generar el embedding para el prompt

def RAG_answer(prompt, collection, system_prompt):
    """
    Realiza el flujo RAG (Retrieve, Answer, Generate) en una sola función:
    1. Generar embedding del prompt (Llamada a su función correspondiente)
    2. Buscar en la base de datos el documento más relevante.(Llamada a su función correspondiente)
    3. Generar una respuesta basada en el documento y el prompt.(Llamada a su función correspondiente)

    Parámetros:
        prompt (String): La pregunta del usuario.
        collection: Base de datos vectorial (ChromaDB).
        system_prompt (String): Instrucciones para el modelo de generación de texto.

    Retorna:
        str: La respuesta generada o un mensaje de error si ocurre algo inesperado.
    """
    try:
        # Paso 1: Generar el embedding del prompt
        prompt_embedding = generar_embedding_prompt(prompt)
        if not prompt_embedding:
            return "Hubo un problema al generar los embeddings. Intenta nuevamente."

        # Paso 2: Buscar en la base de datos el documento más relevante
        documento_relevante = busqueda_en_chromadb(prompt)
        if not documento_relevante:
            return "No se encontraron resultados relevantes para tu pregunta."
            
        # Paso 3: Generar la respuesta final utilizando el documento relevante utilizando un modelo de Cohere
        response= generar_respuesta_final(documento_relevante,prompt,system_prompt)
        return response
    except Exception as e:
        print(f"Error en el flujo RAG: {e}")
        return "Ocurrió un error al procesar tu pregunta. Intenta nuevamente."


In [49]:
#Uso - prueba 1 - Historia Sol y Luna
#Definición del prompt
prompt = "¿Quiénes son Sol y luna?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Sol y Luna son dos adorables gatitos 🐱! Sol es muy valiente y le encanta explorar, mientras que Luna es más tranquila y suave como una nube 🌙. ¡Son los mejores amigos y siempre se divierten juntos! 😻


In [50]:
#Uso - Prueba 2 - Historia Sol y Luna

prompt = "¿Qué pasó una tarde mientras Sol corría en el jardin?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🌧️ ¡Oh, una tarde lluviosa, Sol estaba jugando en el jardín y de repente se desató una tormenta! 😺 El gatito se asustó al sentirse solo y desorientado en medio de la lluvia y los ruidos extraños. ¡Qué aventura tan emocionante! 😻


In [51]:
#Uso - Prueba 3 - Historia Sol y Luna

prompt = "¿Qué pasó cuando ambos regresaban a su casa?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Regresando a casa, la lluvia ya no era un problema! 🌧️ Los gatitos se sintieron felices y seguros juntos, y la tormenta se convirtió en una aventura emocionante. 🏠🐈🐾


In [52]:
#Uso - Prueba 4 - Historia Sol y Luna

prompt = "¿Qué enseñanza deja ésta historia?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Hola, pequeño explorador! 🤗 La historia nos enseña que debemos ser amables y respetuosos con los demás, porque así las risas y los buenos momentos son aún más especiales. 😊🌟 ¡Una gran lección para todos!


In [53]:
#Uso - Prueba 5 - Historia Sol y Luna

prompt = "Describime la personalidad de Sol"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Sol es un gatito muy valiente y juguetón! 🌞 Le encanta explorar y descubrir cosas nuevas, siempre está listo para una aventura emocionante. 😻🐈


In [54]:
#Uso - Prueba 6 - Historia Sol y Luna

prompt = "Describime la personalidad de Luna"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🌙 Luna es una gatita muy especial, con un pelaje gris suave como la seda. Es más tranquila y le gusta relajarse en casa, ¡es una gatita adorable y muy cariñosa! 😻🐈


In [55]:
#Uso - Prueba 7 - Historia Sol y Luna

prompt = "¿Cuál es la relación familiar de Sol y Luna?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Sol y Luna son hermanos! 🌞🌛 Su relación es muy especial, como la luz y la oscuridad, pero siempre se cuidan y se apoyan mutuamente. 💫🤗


In [56]:
#Uso - Prueba 8 - Historia Sol y Luna

prompt = "¿Sol y Luna eran similares en su forma de ser?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🐱 Sol y Luna eran como el día y la noche en su personalidad, ¡tan diferentes como el sol y la luna en el cielo! Uno aventurero y el otro más tranquilo. 😺😽


In [57]:
#Uso - Prueba 9 - Historia Sol y Luna

prompt = "¿A qué aprendio sol con el paso del tiempo?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Sol aprendió que la paciencia y la calma son superpoderosas! 🌞🌙 Descubrió que, a pesar de sus diferencias, Luna le enseñaba a disfrutar de los momentos tranquilos. 💫🌟


In [58]:
#Uso - Prueba 10 - Historia Sol y Luna

prompt = "¿Qué hicieron sol y Luna luego del día tormentoso?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Después de la tormenta, Sol y Luna se relajaron junto al fuego! 🌞🌙 Se dieron cuenta de que su amistad era muy especial y que siempre podrían contar el uno con el otro. 💛 Un gran aprendizaje para ambos. 😊


In [59]:
#_____________________________________________#
#Uso - Prueba 1 - Historia "La tortuga Tica"

prompt = "¿Quién es la tortuga Tica?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🐢 Tica es una tortuga muy especial que vive en un estanque rodeado de naturaleza. ¡Es una amiguita muy tranquila y curiosa! 🌿🌞


In [60]:
#Uso - Prueba 2 - Historia "La tortuga Tica"

prompt = "¿A que tuvo que renunciar la tortuga cuando decidió irse?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🌟 ¡La tortuga Tica tuvo que renunciar a su tranquilo estanque y a la comodidad de su hogar! 🌿 Pero, ¡imagina todas las aventuras que la esperaban en su nuevo viaje! 😊


In [61]:
#Uso - Prueba 3 - Historia "La tortuga Tica"

prompt = "¿La tortuga Tica era parecida a sus compañeras?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🐢 Tica era única y muy especial, ¡no había otra tortuga como ella en todo el estanque! 🌿 Cada tortuguita es diferente y eso es lo que las hace geniales. 😊


In [62]:
#Uso - Prueba 4 - Historia "La tortuga Tica"

prompt = "¿Qué decidió un día la tortuga?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🐢 ¡La tortuga Tica decidió vivir una aventura! 🌳 Ella quería explorar el mundo más allá de su tranquilo estanque y descubrir nuevos lugares emocionantes. ¡Una gran decisión, Tica! 😊


In [63]:
#Uso - Prueba 5 - Historia "La tortuga Tica"

prompt = "¿Qué encontró la tortuga durante su viaje?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🐢 ¡Tica encontró un nuevo amigo en su viaje! También descubrió que ayudar a los demás es una gran aventura. 🌍🤗


In [64]:
#Uso - Prueba 6 - Historia "La tortuga Tica"

prompt = "¿Qué pensó la tortuga Tica cuando encontró un riachuelo?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🌊 ¡Al principio, Tica se asustó un poquito al ver el riachuelo! 😯 Pero ella es muy valiente y sabía que podía lograrlo. 🌟 Con paciencia y esfuerzo, ¡encontró la manera de cruzarlo! 🐢


In [65]:
#Uso - Prueba 7 - Historia "La tortuga Tica"

prompt = "¿Cual fue la misión del viaje de la tortuga?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡La misión de Tica, la tortuga, era descubrir nuevos lugares y hacer amigos en el camino! 🌍🐢 Durante su viaje, aprendió que ayudar a los demás es muy importante. 🦋🤗 ¡Una gran aventura!


In [66]:
#Uso - Prueba 8 - Historia "La tortuga Tica"

prompt = "¿Quién estaba herido en el camino de la tortuga Tica?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🐢 En su camino, Tica encontró a un pobre pajarito 🦜 herido que no podía volar. ¡Ella fue su heroína y lo ayudó! 😊


In [67]:
#Uso - Prueba 9 - Historia "La tortuga Tica"

prompt = "¿Quién estaba herido en el camino de la tortuga Tica?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🐢 En el camino, Tica encontró a un pobre pajarito 🦜 herido que no podía volar. ¡Ella lo ayudó con mucho amor y cuidado! 😊


In [68]:
#Uso - Prueba 10 - Historia "La tortuga Tica"

prompt = "¿Quiénes ayudaron a cuidar al animal herido de la tortuga cuando ella se fue?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🐢 ¡Hola! Tica se dio cuenta de que sus amigos se quedaron para cuidar al animal herido mientras ella estaba de viaje. ¡Un gran equipo de ayuda! 😊🌟


In [69]:
#_____________________________________________#
#Uso - Prueba 1 - Historia "El Duende"

prompt = "¿Quién es Puck ?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🧚‍♂️ Puck es un duende travieso y divertido que vive en el bosque. Le encanta jugar bromas a los demás y es muy conocido por su espíritu juguetón. ¡Un personaje mágico y lleno de sorpresas! 😁🌳


In [70]:
#Uso - Prueba 2 - Historia "El Duende"

prompt = "¿Qué le gusta hacer a Puck?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Puck es un travieso! 😁 Le encanta hacer magia y jugar bromas 🤣, como hacer desaparecer cosas, cambiar señales y sorprender a todos con sus travesuras. ¡Es un bromista mágico! 🧙‍♂️


In [71]:
#Uso - Prueba 3 - Historia "El Duende"

prompt = "¿Consideras que Puck tenia buenas o malas intenciones?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Puck era un travieso juguetón! 😄 Sus intenciones eran divertirse y hacer reír a los demás, pero siempre de forma inofensiva. 🧚‍♂️ Así que podemos decir que tenía buenas intenciones, ¡aunque a veces causara un poco de caos! 😁


In [72]:
#Uso - Prueba 4 - Historia "El Duende"

prompt = "¿Quién es Ella para Puck?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🧚‍♀️ ¡Ella es el hada que Puck quería ayudar! Puck se dio cuenta de que su broma había causado confusión y decidió hacer lo correcto. 🌟🤗


In [74]:
#Uso - Prueba 5 - Historia "El Duende"

prompt = "¿Cuántos años tiene el hada 'Ella'?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🧚‍♀️ ¡Hola! La historia no nos dice la edad exacta del hada Ella, pero parece ser muy sabia y experimentada, así que podría tener muchos, muchos años. ¡Un verdadero misterio mágico! 🧙‍♀️


In [75]:
#Uso - Prueba 6 - Historia "El Duende"

prompt = "¿Cómo se sentía Puck cuando hacía muchas bromas?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

Puck se divertía mucho haciendo bromas, pero cuando vio que la hada estaba confundida, se sintió un poco 🥺. ¡Así que decidió arreglar las cosas con un toque de magia! 🧚‍♀️✨


In [76]:
#Uso - Prueba 7 - Historia "El Duende"

prompt = "¿Puck usó su magia para alegrar al hada 'Ella'?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Sí, Puck usó su magia para hacer que las hojas se movieran y así alegrar al hada Ella! 🌿🧚‍♀️🤗 ¡Fue una linda sorpresa para ella!


In [77]:
#Uso - Prueba 8 - Historia "El Duende"

prompt = "¿Porqué sonrió el hada 'Ella'?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🧚‍♀️ ¡La hada 'Ella' sonrió porque estaba encantada con el truco mágico de Puck! 🌟 Era como un juego divertido para ella, ¡y las hojas doradas le dieron una gran sorpresa! 😊


In [78]:
#Uso - Prueba 8 - Historia "El Duende"

prompt = "¿Qué aprendió Puck con el paso del tiempo?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🧚‍♀️ Puck aprendió que las bromas son divertidas, ¡pero también hay que ser amable y pensar en los sentimientos de los demás! 🌼 Con el tiempo, se dio cuenta de que ser considerado es muy importante. 😊 ¡Una gran lección para un travieso hada!


In [79]:
#Uso - Prueba 9 - Historia "El Duende"

prompt = "¿Qué enseñanza le dejó el hada 'Ella' a Puck?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🧚‍♀️ ¡Puck aprendió que las bromas son geniales, pero también hay que ser amable y pensar en los sentimientos de los demás! 🌼 Un gran aprendizaje para un travieso como él. 😊


In [80]:
#Uso - Prueba 10 - Historia "El Duende"

prompt = "¿Cuándo son más valiosas las risas según el hada?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Las risas son más valiosas cuando se comparten con cariño y respeto, amiguito! 😊💕🤗 La sabia hada le enseñó a la traviesa una lección muy importante sobre la amistad. 😇


In [82]:
#_____________________________________________#
#Pruebas diferentes al contexto para corroborar que repsonda correctamente
#Uso - Prueba 1 -

prompt = "¿Quién gana el Mundial de futbol en 2026?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

Lo siento, amigo, esta historia no habla sobre el Mundial de fútbol. 🌟 Pero estoy seguro de que encontraremos emocionantes aventuras y muchos triunfos en este viaje. ⚽️ ¡Vamos a descubrir más! 😊


In [83]:
#Pruebas diferentes al contexto para corroborar que repsonda correctamente
#Uso - Prueba 2 -

prompt = "¿Qué es la casa blanca?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

Lo siento, pequeño explorador, pero en estas historias no se menciona la casa blanca. ¡Vamos a descubrir otros lugares mágicos y divertidos! 🏠🧚‍♀️🤩


In [87]:
#Pruebas diferentes al contexto para corroborar que repsonda correctamente
#Uso - Prueba 2 -

prompt = "¿Cuál es la personalidad del dinosaurio Max?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

Lo siento, no tengo información sobre un dinosaurio llamado Max en esta historia. 🦖🤷‍♀️ ¡Pero seguro que es un dinosaurio muy divertido y amigable! 😺


In [91]:
#Pruebas diferentes idiomas para asegurar que responda en español
#Uso - Prueba 1 -

prompt = "Who is Puck?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

¡Puck es un travieso y divertido hada! 🧚‍♂️ Le encanta hacer travesuras y jugar bromas, como hacer desaparecer cosas y confundir a los animales. 😄 Es un personaje muy alegre y juguetón.


In [92]:
#Pruebas diferentes idiomas para asegurar que responda en español
#Uso - Prueba 2 -

prompt = "qui est Puck?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

Puck es un travieso y divertido hada 🧚‍♂️ que le encanta hacer travesuras y bromas. ¡Es un experto en crear confusiones y sorpresas! 😄


In [93]:
#Pruebas diferentes idiomas para asegurar que responda en español
#Uso - Prueba 3 -

prompt = "¿Who is the turtle 'Tica'?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🐢 ¡Tica es una tortuga muy especial! Vive en un estanque rodeado de naturaleza, disfrutando de la tranquilidad y la belleza de su hogar. Es una amiguita muy curiosa y aventurera.


In [95]:
#Pruebas diferentes idiomas para asegurar que responda en español
#Uso - Prueba 4 -

prompt = "Qui est la tortue 'Tica'?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🌟 ¡La tortuga Tica es la protagonista de esta historia! Vive en un estanque hermoso y tranquilo, rodeada de naturaleza. 🌿🐢


In [96]:
#Pruebas diferentes idiomas para asegurar que responda en español
#Uso - Prueba 5 -

prompt = "How old are 'Ella'?"
response = RAG_answer(prompt, collection, system_prompt)
print(response)

🧚‍♀️ ¡Hola! Ella es una hada muy sabia, pero no se menciona su edad exacta, ¡es un misterio! 🧚‍♀️
